# Kaggle League of Legends competition - Neural Network Models

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

## How to Win at League of Legends?

### Uninstall LoL and [install Dota 2](https://store.steampowered.com/app/570/Dota_2/), EZ. (just kidding, both games are great. Volvo pls gib patch.)

<img src = "https://static.wikia.nocookie.net/dota2_gamepedia/images/7/78/Keyart_phoenix.jpg/revision/latest/" alt="SKREE CAW CAW IM A BIRD" style="width:800px;height:497px;">

In [32]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold

from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.preprocessing import MinMaxScaler

In [5]:
X_train_original = pd.read_csv('../data/participants_train.csv')
X_test_original = pd.read_csv('../data/participants_test.csv')
y_train_original = pd.read_csv('../data/train_winners.csv')

champion_mastery = pd.read_csv('../data/champion_mastery.csv')
champion = pd.read_json('../data/champion.json')

team_positions = pd.read_csv('../data/teamPositions.csv')

train_last_frame_values = pd.read_csv('../data/train_last_frame_values.csv')
test_last_frame_values = pd.read_csv('../data/test_last_frame_values.csv')

training_events = pd.read_csv('../data/training_events.csv')
testing_events = pd.read_csv('../data/testing_events.csv')

submission = pd.read_csv('../data/sample_submission.csv')

In [6]:
# function that converts values to negative (for the second team, teamId 200)
# it leaves the first team values, teamId intact

def convert_team_values(df, col_names):
    
    
    for col in col_names:
        df[col] = np.where(df['teamId'] == 200,
                            -1* df[col],
                                df[col])
        
    return

In [7]:
# transformation needed on the dataframes (see the file prediction_models)
vars = ['wards_placed', 'wards_killed', 'turretplates_destroyed', 'elite_monsters_killed']

convert_team_values(training_events, vars)
convert_team_values(testing_events, vars)

training_events = training_events.groupby('matchId')[vars].sum()
testing_events = testing_events.groupby('matchId')[vars].sum()

champion_data = pd.json_normalize(champion['data'])
champion_data['key'] = champion_data['key'].astype(int)

champion_types= champion_data.explode('tags').pivot_table(values='id', index='key', columns='tags', aggfunc='count').fillna(0).reset_index()

In [8]:
# to be used later to measure the accuracy!
kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)

# this is to extract the column that is needed for training
y_train = y_train_original['winner']

## Using Neural Networks now to try and improve the logreg predictions!

In [9]:
variables = ['summonerLevel', 'championLevel','championPoints', 
             'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ]

X_train = pd.merge(X_train_original, team_positions, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_data, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, champion_types, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, train_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_train, variables)

X_train_perlane = X_train.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_train_perlane[f'{lane}'] = np.where(X_train_perlane[f'{lane}'] >= 0, 1, -1)

X_train = (
    X_train
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_train = pd.merge(X_train, X_train_perlane, how='inner', on='matchId').reset_index(drop = True)
X_train = pd.merge(X_train, training_events, how='inner', on='matchId').reset_index(drop = True)

In [10]:
X_train.head()

,matchId,summonerLevel,championLevel,championPoints,Assassin,Fighter,Mage,Marksman,Support,Tank,...,final_enemycontrolled,BOTTOM,JUNGLE,MIDDLE,TOP,UTILITY,wards_placed,wards_killed,turretplates_destroyed,elite_monsters_killed
0,0,682,0.0,-605428.0,-1.0,-1.0,2.0,0.0,0.0,-2.0,...,67664,-1,-1,1,-1,-1,-64,-1,-2,-1
1,1,628,8.0,1356027.0,3.0,0.0,0.0,0.0,-1.0,-1.0,...,-61783,1,1,1,-1,1,1,0,-1,-1
2,2,1049,1.0,-273911.0,-2.0,1.0,0.0,0.0,0.0,0.0,...,-132630,1,-1,-1,-1,-1,4,1,-2,1
3,3,-1027,-3.0,-287667.0,1.0,1.0,1.0,-1.0,-1.0,0.0,...,-39616,1,1,-1,-1,1,4,0,2,0
4,4,1612,7.0,503668.0,0.0,0.0,2.0,-1.0,0.0,-1.0,...,16629,-1,1,-1,1,1,4,0,-1,-1


In [11]:
pipeline_neuralnetwork = Pipeline(
    steps = [
        ('scaler', MinMaxScaler()),
        ('nn', MLPClassifier(verbose = True,
                             hidden_layer_sizes = (100, 100, 100, 100),
                             activation = 'tanh',
                             max_iter = 10000,
                             alpha=0.05))
    ]
)

In [12]:
pipeline_neuralnetwork.fit(X_train, y_train)

Iteration 1, loss = 0.67293528
Iteration 2, loss = 0.62760996
Iteration 3, loss = 0.61507694
Iteration 4, loss = 0.60792764
Iteration 5, loss = 0.60942378
Iteration 6, loss = 0.60098059
Iteration 7, loss = 0.59904177
Iteration 8, loss = 0.59567799
Iteration 9, loss = 0.59229353
Iteration 10, loss = 0.59068142
Iteration 11, loss = 0.60015370
Iteration 12, loss = 0.60409990
Iteration 13, loss = 0.59377210
Iteration 14, loss = 0.59273640
Iteration 15, loss = 0.58455900
Iteration 16, loss = 0.58548125
Iteration 17, loss = 0.58130663
Iteration 18, loss = 0.58291796
Iteration 19, loss = 0.59040405
Iteration 20, loss = 0.58682680
Iteration 21, loss = 0.58339420
Iteration 22, loss = 0.58146511
Iteration 23, loss = 0.58046965
Iteration 24, loss = 0.57775138
Iteration 25, loss = 0.57732240
Iteration 26, loss = 0.57702880
Iteration 27, loss = 0.57357318
Iteration 28, loss = 0.57858250
Iteration 29, loss = 0.57253075
Iteration 30, loss = 0.57304390
Iteration 31, loss = 0.57745479
Iteration 32, los

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('nn',
                 MLPClassifier(activation='tanh', alpha=0.05,
                               hidden_layer_sizes=(100, 100, 100, 100),
                               max_iter=10000, verbose=True))])

In [13]:
accuracy_score(
    y_true = y_train,
    y_pred = pipeline_neuralnetwork.predict(X_train)
)

0.7315

In [14]:
summoner_cv_scores = cross_val_score(
    estimator = pipeline_neuralnetwork,
    X = X_train,
    y = y_train,
    cv = kfold
)

print(summoner_cv_scores)
print(np.mean(summoner_cv_scores))

Iteration 1, loss = 0.68012126
Iteration 2, loss = 0.63318414
Iteration 3, loss = 0.62492725
Iteration 4, loss = 0.61127346
Iteration 5, loss = 0.60608691
Iteration 6, loss = 0.60779001
Iteration 7, loss = 0.59552473
Iteration 8, loss = 0.59335532
Iteration 9, loss = 0.60281433
Iteration 10, loss = 0.59165388
Iteration 11, loss = 0.58819519
Iteration 12, loss = 0.59135862
Iteration 13, loss = 0.60295727
Iteration 14, loss = 0.58716376
Iteration 15, loss = 0.58519290
Iteration 16, loss = 0.58470119
Iteration 17, loss = 0.58602775
Iteration 18, loss = 0.58489872
Iteration 19, loss = 0.58357934
Iteration 20, loss = 0.58700730
Iteration 21, loss = 0.58553890
Iteration 22, loss = 0.57925451
Iteration 23, loss = 0.57869636
Iteration 24, loss = 0.57895957
Iteration 25, loss = 0.58066444
Iteration 26, loss = 0.58006968
Iteration 27, loss = 0.57487156
Iteration 28, loss = 0.57452987
Iteration 29, loss = 0.57986518
Iteration 30, loss = 0.57165764
Iteration 31, loss = 0.57520205
Iteration 32, los

In [15]:
variables = ['summonerLevel', 'championLevel','championPoints', 
             'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ]

X_test = pd.merge(X_test_original, team_positions, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_data, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, champion_types, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, test_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_test, variables)

X_test_perlane = X_test.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_test_perlane[f'{lane}'] = np.where(X_test_perlane[f'{lane}'] >= 0, 1, -1)

X_test = (
    X_test
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_test = pd.merge(X_test, X_test_perlane, how='inner', on='matchId').reset_index(drop = True)
X_test = pd.merge(X_test, testing_events, how='inner', on='matchId').reset_index(drop = True)

In [16]:
y_pred = pipeline_neuralnetwork.predict(X_test)

In [17]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,200
2,8002,200
3,8003,200
4,8004,200


In [18]:
submission.to_csv('../data/submission_neural_network_hidden_layers4x100_alpha005_activtahn_2023_03_30.csv', index=False)

### Attempt at gradient boosting

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [52]:
X_train = X_train_original

X_train = pd.get_dummies(X_train, prefix='Champion')
champions_encoded = list(X_train.drop(columns=['matchId', 'teamId', 'participantId', 'summonerId', 'summonerLevel', 'championId']).columns.values)

variables = ['summonerLevel', 'championLevel','championPoints', 
            # 'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            # 'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            # 'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            # 'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ] + champions_encoded

X_train = pd.merge(X_train, team_positions, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_data, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, champion_types, how='inner', left_on='championId', right_on='key')
X_train = pd.merge(X_train, train_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_train = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_train = X_train.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_train, variables)

X_train_perlane = X_train.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_train_perlane[f'{lane}'] = np.where(X_train_perlane[f'{lane}'] >= 0, 1, -1)

X_train = (
    X_train
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_train = pd.merge(X_train, X_train_perlane, how='inner', on='matchId').reset_index(drop = True)
X_train = pd.merge(X_train, training_events, how='inner', on='matchId').reset_index(drop = True)

In [53]:
gbr = Pipeline(
    steps = [
        #('scaler', MinMaxScaler()),
        ('gb', GradientBoostingClassifier(n_estimators = 1000, learning_rate=0.01))
    ]
)

gbr.fit(X_train, y_train)

Pipeline(steps=[('gb',
                 GradientBoostingClassifier(learning_rate=0.01,
                                            n_estimators=1000))])

In [54]:
accuracy_score(
    y_true = y_train,
    y_pred = gbr.predict(X_train)
)

0.76125

In [55]:
print(classification_report(y_train, gbr.predict(X_train)))

              precision    recall  f1-score   support

         100       0.76      0.77      0.77      4071
         200       0.76      0.75      0.76      3929

    accuracy                           0.76      8000
   macro avg       0.76      0.76      0.76      8000
weighted avg       0.76      0.76      0.76      8000



In [48]:
importances = pd.DataFrame({
    'variable': gbr.feature_names_in_,
    'importance': gbr['gb'].feature_importances_
})

importances.sort_values('importance', ascending = False).head(10)

,variable,importance
10,final_gold,0.609183
227,elite_monsters_killed,0.045338
11,final_xp,0.044460
2,championLevel,0.033303
67,Champion_AurelionSol,0.018480
23,final_health,0.014638
43,final_dmgdone,0.013148
1,summonerLevel,0.012556
219,BOTTOM,0.010200
64,Champion_Annie,0.009200


In [49]:
X_test = X_test_original

X_test = pd.get_dummies(X_test, prefix='Champion')
champions_encoded = list(X_test.drop(columns=['matchId', 'teamId', 'participantId', 'summonerId', 'summonerLevel', 'championId']).columns.values)

variables = ['summonerLevel', 'championLevel','championPoints', 
            # 'Assassin', 'Fighter', 'Mage', 'Marksman', 'Support', 'Tank',
            # 'info.attack', 'info.defense', 'info.magic', 'info.difficulty',
            # 'stats.hpregenperlevel',	'stats.mpregen', 'stats.mpregenperlevel',	'stats.crit',	'stats.critperlevel',
            # 'stats.attackdamage', 'stats.attackdamageperlevel', 'stats.attackspeedperlevel',	'stats.attackspeed',
            'final_gold', 'final_xp', 'final_abilityhaste', 'final_abilitypower', 'final_armor', 'final_armorpen',
            'final_armorpenpercent', 'final_atkdmg', 'final_bns_armorpenpercent', 'final_bns_magicpenpercent', 'final_ccreduction',
            'final_cdreduction', 'final_remaining_health', 'final_health', 'final_healthrgn', 'final_lifesteal', 'final_mppen',
            'final_mgpenpercent', 'final_mgres', 'final_ms', 'final_omnivamp', 'final_physicalvamp', 'final_power', 'final_powermax',
            'final_powerregen', 'final_spellvamp', 'final_currentgold', 'final_magicdmgdone', 'final_magicdmgdonetochamps', 'final_magicdmgtaken',
            'final_physdmgdone', 'final_physdmgdonetochamps', 'final_physdmgtaken', 'final_dmgdone', 'final_dmgdonetochamps', 'final_dmgtaken', 
            'final_truedmgdone', 'final_truedmgdonetochamps', 'final_truedmgtaken', 'final_goldpersec', 'final_jungleminionskilled', 'final_lvl',
            'final_minionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_jungleminionskilled', 'final_enemycontrolled'                  
             ] + champions_encoded

X_test = pd.merge(X_test, team_positions, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_data, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, champion_types, how='inner', left_on='championId', right_on='key')
X_test = pd.merge(X_test, test_last_frame_values, how='inner', on=['matchId', 'participantId'])
X_test = pd.merge(X_test, champion_mastery, how='left', on=['summonerId', 'championId']).fillna(0)

X_test = X_test.sort_values(['matchId', 'participantId'], ascending = [True, True]).reset_index(drop=True)


convert_team_values(X_test, variables)

X_test_perlane = X_test.groupby(['matchId', 'teamPosition'])[['final_gold']].sum().pivot_table(values='final_gold', index='matchId', columns='teamPosition').reset_index().drop(columns=0)

for lane in ['BOTTOM', 'JUNGLE', 'MIDDLE', 'TOP', 'UTILITY']:
  X_test_perlane[f'{lane}'] = np.where(X_test_perlane[f'{lane}'] >= 0, 1, -1)

X_test = (
    X_test
    .groupby(['matchId'])[variables]
    .sum()
    .reset_index()
)

X_test = pd.merge(X_test, X_test_perlane, how='inner', on='matchId').reset_index(drop = True)
X_test = pd.merge(X_test, testing_events, how='inner', on='matchId').reset_index(drop = True)

In [50]:
y_pred = gbr.predict(X_test)

In [51]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,200
2,8002,200
3,8003,200
4,8004,200


In [42]:
# best submission so far 7.08 !
#submission.to_csv('../data/submission_gradientboosting_n1000_learn001_2023_04_01.csv', index=False)

# submission with this (all champs) is 7.06
#submission.to_csv('../data/submission_gradientboosting_all_champions_n1000_learn001_2023_04_01.csv', index=False)